# 1stm

In [24]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [25]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [26]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [27]:
nrun = 1000

1000

In [28]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1061]
        t2 = t[:, 1062:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [29]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1382.62,"[-0.288356, -0.776062, -0.560873]",178.482,"[0.618983, 2.25688, -3.44101]",0.540531,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 12.94 52.23 58.83 |"
2,1/complex.2.pdb,1268.85,"[-0.666085, -0.694759, -0.271367]",176.791,"[-22.9733, 10.6354, 29.1604]",0.0739988,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 26.14 56.70 79.31 |"
3,1/complex.3.pdb,1220.26,"[7.41491e-5, 0.964482, -0.26415]",69.9221,"[23.4389, 10.597, 38.699]",-12.3311,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 26.55 57.99 15.74 |"
4,1/complex.4.pdb,1148.32,"[0.810048, -0.0792923, 0.580978]",92.9537,"[-10.3575, 71.09, 24.1436]",-13.9145,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 4.47 97.00 41.37 |"
5,1/complex.5.pdb,1132.07,"[0.723961, -0.219214, 0.654085]",90.5074,"[-0.910044, 50.7296, 18.0091]",18.6965,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 13.84 61.56 68.20 |"
6,1/complex.6.pdb,1115.83,"[-0.294075, 0.908303, -0.2975]",66.8091,"[35.4869, 24.5698, 39.9362]",-13.3557,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 27.42 53.99 16.40 |"
7,1/complex.7.pdb,1108.46,"[0.849514, -0.408811, -0.333467]",173.921,"[29.9593, 49.5108, 15.6244]",0.393369,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 13.38 32.71 17.41 |"
8,1/complex.8.pdb,1102.83,"[-0.644808, 0.413085, -0.643105]",175.313,"[10.819, 58.0152, 26.4171]",16.9155,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … -15.05 44.73 26.11 |"
9,1/complex.9.pdb,1089.06,"[-0.771621, 0.403599, -0.49164]",87.4786,"[11.4216, 68.3426, 38.178]",-16.978,"1x2122 TrjArray{Float64, Int64}\n| 20.91 51.89 13.34 | … 26.01 35.08 37.19 |"


In [30]:
df_org = deepcopy(df);

In [31]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [32]:
#using BSON: @save, @load
#@save "df.bson" df

In [33]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)

In [34]:
df;